In [1]:
import os
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
from helper.utils import load_from_pickle, load_from_shelve
import numpy as np
import pandas as pd
from plotly import tools
import plotly as py
import plotly.graph_objs as go
#import impywidgets as widget
py.offline.init_notebook_mode(connected=True)

/home/jackalhan/Development/github/more_meaningful_representations/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
base_data_path = "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/"

In [12]:
debug_files = {}
for i, structure in enumerate(os.walk(base_data_path)):
    root, dirs, files = structure
    for file in files:
        if file.endswith(".pkl"):            
            debug_files[i] = os.path.join(root, file)
["{} --> {}".format(key, value) for key, value in debug_files.items()]

['1 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_9_epoch_120_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_1024_1024_1024_1024_1024_1024_1024_1024_1024_keep_1_1_1_1_1_1_1_1_1_seed_66_66_66_66_66_66_66_66_66_loss_v3/debug_dict.pkl',
 '2 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_6_epoch_120_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_2048_2048_2048_2048_2048_2048_keep_1_1_1_1_1_1_seed_66_66_66_66_66_66_loss_v3/debug_dict.pkl']

In [13]:
model_indx =1
my_dict = load_from_pickle(os.path.join(base_data_path, 
                                        debug_files[model_indx]))
num_of_epochs = my_dict['epochs']
number_of_sample_question_size = 15
# temp_ids = np.arange(my_dict['actual_labels'].shape[0])
# np.random.shuffle(temp_ids)
# question_ids = temp_ids[:number_of_sample_question_size]
debug_files[model_indx]

'/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_9_epoch_120_mar_0.6_sf_0.1_0.1_0.1_0.1_0.1_0.1_0.1_0.1_0.1_wd_0.001_0.001_0.001_0.001_0.001_0.001_0.001_0.001_0.001_lr_0.001_dim_1024_1024_1024_1024_1024_1024_1024_1024_1024_keep_1_1_1_1_1_1_1_1_1_seed_66_66_66_66_66_66_66_66_66_loss_v3/debug_dict.pkl'

In [14]:
"""
How the recalls at top n are changing for each epoch 
"""

data_recalls = []
sub_list = []
sub_list.append(0)
sub_list.extend([x for x in my_dict['normalized_recalls_before_model']])
data_recalls.append(sub_list)
for i in range(1,num_of_epochs+1):
    sub_list = []
    sub_list.append(i)
    sub_list.extend([x for x in my_dict[str(i)]['normalized_recalls_after_model']])
    data_recalls.append(sub_list)
df_data_recalls = pd.DataFrame(data_recalls, columns=['epoch', 'top_1', 'top_2', 'top_5'
                                                   ,'top_10', 'top_20', 'top_50'])
traces = []
for c in df_data_recalls.columns[1:]:
    trace = go.Scatter(
    x = df_data_recalls.index,
    y = df_data_recalls[c],
    mode = 'lines+markers',
    name = 'Top {}'.format(c))
    traces.append(trace)

layout = go.Layout(
    title='How the recalls at top n are changing for each epoch',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Recall Values'
    )
)
fig =  go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [15]:
df_data_recalls[['top_1', 'top_2', 'top_5','top_10', 'top_20', 'top_50']].mean(axis=1).sort_values(ascending =[False])

51     0.719116
50     0.719072
47     0.718986
48     0.718942
46     0.718942
49     0.718899
59     0.718856
54     0.718856
53     0.718856
52     0.718856
61     0.718856
72     0.718856
73     0.718812
56     0.718812
45     0.718769
71     0.718769
70     0.718726
74     0.718726
55     0.718726
62     0.718726
57     0.718726
60     0.718682
58     0.718682
68     0.718639
69     0.718639
76     0.718639
40     0.718596
77     0.718552
65     0.718552
66     0.718552
         ...   
112    0.717208
105    0.717165
12     0.717165
107    0.717165
29     0.717165
28     0.717165
27     0.716992
25     0.716732
26     0.716688
22     0.716515
24     0.716515
23     0.716385
8      0.716342
20     0.716168
21     0.716081
17     0.715995
13     0.715995
18     0.715951
19     0.715951
16     0.715908
14     0.715778
15     0.715605
7      0.712050
6      0.702427
0      0.697096
5      0.684222
4      0.641006
3      0.536844
2      0.323017
1      0.105505
Length: 121, dtype: floa

In [16]:
df_data_recalls.sort_values(by=['top_1', 'top_2', 'top_5', 'top_10', 'top_20', 'top_50'], ascending =[False, False, False, False, False, False])

,epoch,top_1,top_2,top_5,top_10,top_20,top_50
46,46,0.475683,0.590637,0.705592,0.787516,0.850455,0.903771
47,47,0.475683,0.590377,0.705852,0.787516,0.850455,0.904031
45,45,0.475423,0.590637,0.704811,0.786996,0.850455,0.904291
48,48,0.474902,0.590377,0.706112,0.788036,0.850455,0.903771
44,44,0.474902,0.590117,0.704551,0.786996,0.849935,0.904291
49,49,0.474902,0.589857,0.705852,0.788557,0.850455,0.903771
12,12,0.474902,0.583095,0.705072,0.784655,0.848635,0.906632
42,42,0.474642,0.589857,0.703771,0.787776,0.850195,0.905072
14,14,0.474642,0.581274,0.702211,0.783095,0.848114,0.905332
13,13,0.474642,0.580234,0.703251,0.782835,0.848635,0.906372


In [11]:
"""
which of questions are getting worse (moving farther from its ground-truth) through subsequent epochs.
"""

k = 1
good_questions_at_top_1_idx = np.where(my_dict['top_k'][str(k)]['are_founds_before'] == 1)
np.random.shuffle(good_questions_at_top_1_idx[0])
# IN ORDER TO SELECT RANDOM QUESTIONS
#question_ids = good_questions_at_top_1_idx[0][:number_of_sample_question_size]

# IN ORDER TO ENTER QUESTIONS
question_ids = [1550, 4257, 459]

good_questions_at_top_1_are_founds = my_dict['top_k'][str(k)]['are_founds_before'][question_ids]
good_questions_at_top_1_labels = my_dict['top_k'][str(k)]['closest_labels_before'][question_ids]
good_questions_at_top_1_distances = my_dict['top_k'][str(k)]['distances_before'][question_ids]

still_good_question = pd.DataFrame()
still_good_question[0]= np.reshape(good_questions_at_top_1_are_founds, [-1,])
for i in range(1,num_of_epochs+1):
    still_good_question[i] = np.reshape(my_dict[str(i)]['top_k'][str(k)]['are_founds_after'][question_ids], [-1,])
still_good_question = still_good_question.T
still_good_question.columns = [question_ids[col_name] for col_name in still_good_question.columns]
data = []
problematic_questions = []
for q in question_ids:
    _problematic_question = np.where(still_good_question[q] == 0)[0]
    _problematic_question = still_good_question[q][_problematic_question]
    if _problematic_question.shape[0] != 0:
        problematic_questions.append(_problematic_question.name)
    trace = go.Scatter(
    x = still_good_question.index,
    y = still_good_question[q],
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Questions" Paragraphs are still in their top {} closest neighbours'.format(k),
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Yes or No'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)
print('Problematic Questions')
[_p for _p in problematic_questions]

IndexError: index 4257 is out of bounds for axis 0 with size 3845

In [12]:
"What happened to the following question?"
sample_question_that_are_getting_worse_idx = problematic_questions
sample_question_that_are_getting_worse_sub_idx = [np.where(question_ids==q)[0][0] for q in sample_question_that_are_getting_worse_idx]
#label_of_sample_question = good_questions_at_top_1_labels[sample_question_that_are_getting_worse_sub_idx]
"Is that question getting farther from its ground-truth paragraph?"
getting_worse_distances = pd.DataFrame()
getting_worse_labels = pd.DataFrame()
getting_worse_distances[0] = np.reshape(good_questions_at_top_1_distances[sample_question_that_are_getting_worse_sub_idx], [-1,])
getting_worse_labels[0] = np.reshape(good_questions_at_top_1_labels[sample_question_that_are_getting_worse_sub_idx], [-1,])
for i in range(1,num_of_epochs+1):
    getting_worse_distances[i] = np.reshape(my_dict[str(i)]['top_k'][str(1)]['distances_after'][sample_question_that_are_getting_worse_idx], [-1,])
    getting_worse_labels[i] = np.reshape(my_dict[str(i)]['top_k'][str(1)]['closest_labels_after'][sample_question_that_are_getting_worse_idx], [-1,])

getting_worse_distances = getting_worse_distances.T
getting_worse_labels = getting_worse_labels.T
getting_worse_distances.columns = [sample_question_that_are_getting_worse_idx[i] for i, col_name in enumerate(getting_worse_distances.columns)]
getting_worse_labels.columns = [sample_question_that_are_getting_worse_idx[i] for i, col_name in enumerate(getting_worse_labels.columns)]

data = []
for q in sample_question_that_are_getting_worse_idx:
    trace = go.Scatter(
    x = getting_worse_distances.index,
    y = getting_worse_distances[q],
    text = getting_worse_labels[q],
    hoverinfo = 'y' 'text',
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Are questions getting farther from their ground-truth paragraph in each epoch iteration? <br> ' +
    'to grab the closest paragraph, please hover the question',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Distance to Closest Paragraph'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)
closest_paragraphs_of_problematic_questions = dict()
print('Closest Paragraphs of these Problematic Questions')
for _q in problematic_questions:
    closest_paragraphs_of_problematic_questions[_q] = set(getting_worse_labels[_q]) 
closest_paragraphs_of_problematic_questions

NameError: name 'problematic_questions' is not defined

In [13]:
question_id_to_examine = 4257

print('Question {} is displayed with itsprogress within all epoch and k values'.format(question_id_to_examine))
question_to_display_progress = df_traced_questions[df_traced_questions['question_id'] == question_id_to_examine]
# question_to_display_progress

Question 4257 is displayed with itsprogress within all epoch and k values


NameError: name 'df_traced_questions' is not defined

In [14]:
print('Question {} is plotted'.format(question_id_to_examine))
paragraphs_to_plot = set(question_to_display_progress['paragraph_id'])
data = []
for p in paragraphs_to_plot:
    df = question_to_display_progress[question_to_display_progress['paragraph_id'] == p].sort_values(by=['epoch', 'k'], ascending=[True, True])
    if df.shape[0] > num_of_epochs:
        name = 'Ground Truth {}'.format(p)
    else:
        name = 'Paragraph {}'.format(p)
    df = df.drop_duplicates(subset=['question_id', 'epoch'])
    plot_data = []
    for _epoch in range(num_of_epochs + 1):
        try:
            distance = df[df['epoch'] == _epoch]['distance_to_paragraph'].values[0]
        except:
            distance = 1.5
        plot_data.append((_epoch, distance))
    plot_df = pd.DataFrame(plot_data, columns=['epoch', 'distance'])  
    trace = go.Scatter(
    x = plot_df.index,
    y = plot_df['distance'],
    mode = 'lines+markers',
    name = name)
    data.append(trace)

layout = go.Layout(
    title='Distances of the paragraphs to that question {} are getting progressed in each epoch'.format(question_id_to_examine),
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Distance'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

Question 4257 is plotted


NameError: name 'question_to_display_progress' is not defined

In [15]:
"""
What is the progress of the paragraphs in which they became a ground-truth of another questions?
# First we need to find which questions were assigned to that paragraph
"""

paragraph_ids_to_examine = [15135]
questions = []
for p in paragraph_ids_to_examine:
    try:
        print('*' * 10)
        print('Parahraph {}'.format(p)) 
         # FOR EPOCH 0 : BEFORE APPENDIX
        _epoch = 0
        for _k in range(total_ks):
            try:
                question = np.where(data_dict['top_k'][str(_k+1)]['closest_labels_before'] 
                                                    == p)[0][_k]
                paragraph_at_k = my_dict['top_k'][str(total_ks)]['closest_labels_before'][question][_k]
                q_to_p_distance_at_k = my_dict['top_k'][str(total_ks)]['distances_before'][question][_k]
                is_ground_truth_in_k_range = my_dict['top_k'][str(_k+1)]['are_founds_before'][question][0]
                questions.append((question,
                                 _epoch,
                                 _k+1,
                                 paragraph_at_k, 
                                 q_to_p_distance_at_k,
                                 is_ground_truth_in_k_range
                ))
            except:
                print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))

        # FOR EPOCH > 0 : AFTER APPENDIX
        for _epoch in range(1,num_of_epochs+1):
            for _k in range(total_ks):
                try:
                    paragraph_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['closest_labels_after'][question][_k]
                    q_to_p_distance_at_k = my_dict[str(_epoch)]['top_k'][str(total_ks)]['distances_after'][question][_k]
                    is_ground_truth_in_k_range = my_dict[str(_epoch)]['top_k'][str(_k+1)]['are_founds_after'][question][0]
                    questions.append((question,
                                     _epoch,
                                     _k+1,
                                     paragraph_at_k, 
                                     q_to_p_distance_at_k,
                                     is_ground_truth_in_k_range
                    ))
                except:
                    print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))
                
    except:
        print('There is no such a question that belongs to paragraph {} in that 5000 subset at epoch:{}, k:{}'.format(p, _epoch, _k+1))

df_traced_questions = pd.DataFrame(questions, columns=['question_id','epoch', 'k', 'paragraph_id', 'distance_to_paragraph', 'is_ground_truth_in_k_range'])   

**********
Parahraph 15135


NameError: name '_k' is not defined

In [16]:
"""
In each training iteration, Are questions moving closer to ground_truth or not
if a delta value is moving to smaller value, it means question is moving to farther from its ground-truth, 
else is moving to closer to its ground-truth. The bigger, the better.
"""

data_delta = pd.DataFrame()
for i in range(1,num_of_epochs+1):
    data_delta[i] = my_dict[str(i)]['delta_before_after_model'][question_ids]
data_delta = data_delta.T
data_delta.columns = [question_ids[col_name] for col_name in data_delta.columns]
data = []
for q in question_ids:
    trace = go.Scatter(
    x = data_delta.index,
    y = data_delta[q],
    mode = 'lines+markers',
    name = 'Question {}'.format(q))
    data.append(trace)

layout = go.Layout(
    title='Delta Before Trained Q. Embeds. to Ground Truth vs After Trained Q. Embeds. to Ground Truth',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Delta Distance'
    )
)
fig =  go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

IndexError: index 4257 is out of bounds for axis 1 with size 3845

In [17]:
"""
Standart Deviation of the distances q to ground truths for each epoch. If it is getting smaller, which means that 
embeddings are getting look like each other
"""

data_dev = []
data_dev.append(np.std(my_dict['distance_from_before_model_q_to_p']))
for i in range(1,num_of_epochs+1):
    data_dev.append(np.std(my_dict[str(i)]['distance_from_after_model_q_to_p']))
df_dev = pd.DataFrame(data=data_dev, columns=['std_dev'])
trace = go.Scatter(
x = df_dev.index,
y = df_dev['std_dev'],
mode = 'lines+markers')

layout = go.Layout(
    title='Standart Deviation of the distances q to ground truths for each epoch',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Standart Deviation of the distances'
    )
)
fig =  go.Figure(data=[trace], layout=layout)
py.offline.iplot(fig)

In [18]:
"""
The idea here is that how many pairs we have that their distance are getting worse means that
they are getting farther. 
"""
number_of_bad_questions = []
set_bad_questions = set()
for i in range(1,num_of_epochs+1):
    bad_questions = np.where(my_dict[str(i)]['distance_from_after_model_q_to_p'] < 0)
    bad_questions = my_dict[str(i)]['distance_from_after_model_q_to_p'][bad_questions]
    number_of_bad_questions.append(bad_questions.shape[0])
    set_bad_questions.update(bad_questions)
    
df_farther = pd.DataFrame(data=number_of_bad_questions, columns=['number_of_bad_questions'])
trace = go.Scatter(
x = df_farther.index,
y = df_farther['number_of_bad_questions'],
mode = 'lines+markers')

layout = go.Layout(
    title='How many questions are getting farther from its pair paragraph',
    xaxis=dict(
        title='Epochs'
    ),
    yaxis=dict(
        title='Number of bad questions'
    )
)
fig =  go.Figure(data=[trace], layout=layout)
py.offline.iplot(fig)